In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import sys
import logging
from pprint import pprint

from sklearn.model_selection import KFold

sys.path.append(Path('.').absolute().parent.resolve().as_posix())
sys.path.append((Path('.').absolute().parent / 'source').resolve().as_posix())
from data_loader import config_loader, data_preprocessing
from logging_util.logger import get_logger
from models import estimators
from evaluation import evaluation

config = config_loader.load_config()
logger = get_logger(__name__)
logging.getLogger("PIL").setLevel(logging.WARNING)
logging.getLogger('matplotlib').setLevel(logging.WARNING)

2024-01-12 10:04:20,919 - DEBUG - config_loader.py:18 - Loading data configuration
2024-01-12 10:04:20,929 - INFO - __init__.py:19 - Loading HistGradientBoostRegressor
2024-01-12 10:04:21,238 - DEBUG - __init__.py:350 - matplotlib data path: c:\Users\steve\anaconda3\envs\fertilizer_usage\lib\site-packages\matplotlib\mpl-data
2024-01-12 10:04:21,245 - DEBUG - __init__.py:350 - CONFIGDIR=C:\Users\steve\.matplotlib
2024-01-12 10:04:21,249 - DEBUG - __init__.py:1511 - interactive is False
2024-01-12 10:04:21,250 - DEBUG - __init__.py:1512 - platform is win32
2024-01-12 10:04:21,378 - DEBUG - __init__.py:350 - CACHEDIR=C:\Users\steve\.matplotlib
2024-01-12 10:04:21,389 - DEBUG - font_manager.py:1574 - Using fontManager instance from C:\Users\steve\.matplotlib\fontlist-v330.json
2024-01-12 10:04:22,340 - DEBUG - config_loader.py:18 - Loading data configuration
2024-01-12 10:04:22,344 - DEBUG - config_loader.py:18 - Loading data configuration
2024-01-12 10:04:22,348 - INFO - __init__.py:19 - 

In [2]:
estimators

{'HistGradientBoostRegressor': models.HistGradientBoostRegressor.estimator.HistGradientBoostEstimator,
 'XgboostRegressor': models.XgboostRegressor.estimator.XGBoostEstimator}

In [3]:
target = 'K2O_avg_app'
config.target = target

In [4]:
resultspath = Path().resolve().parent.resolve() / 'results' / target
resultspath

WindowsPath('C:/Users/steve/OneDrive - Universiteit Antwerpen/Documenten/00IDLab/projects/crop_database_imputation/results/K2O_avg_app')

In [5]:
# TODO: add type hints

def ohe(input_data, config):
    
    datapath = Path().resolve().parent.resolve() / 'data'
    dtype = data_preprocessing.get_data_types(config)
    all_data = pd.read_csv(f'{datapath}/{config.csv_files["features"]}', dtype=dtype, usecols=dtype.keys()) 

    input_data_ohe = data_preprocessing.one_hot_encoding(input_data)

    return input_data_ohe

def load_input_data(resultspath: Path, it: int = 0, test: bool = False, config=config):
    dtype = data_preprocessing.get_data_types(config)
    X = pd.read_csv(resultspath / f'data/{"test" if test else "train"}_set_{it}.csv', index_col=0, dtype=dtype)
    return X

def make_prediction(estimator, X, config):
    # drop the last column, which is the target
    if config.target in X.columns:
        X = X.drop(config.target, axis=1)
    X = ohe(X, config)
    y_pred = pd.Series(
            estimator.model.predict(X),
            index=X.index,
            name='predicted_' + config.target,
        )
    return y_pred

for estimator_name, estimator_cls in estimators.items():
    print(estimator_name)
    estimator = estimator_cls(outpath=resultspath)
    estimator = estimator.load(path=estimator.output_path, it=0)
    X = load_input_data(resultspath, it=0, test=True, config=config)    
    y_pred = make_prediction(estimator, X, config)

    # check if the predictions match
    print((y_pred != estimator.y_pred).any())



HistGradientBoostRegressor
False
XgboostRegressor


ValueError: feature_names mismatch: ['Year', 'Area_report', 'Fert_perc', 'Area_fert', 'Area_FAOStat', 'Perc_Agr_Area_FAOStat', 'GDP_Capita_WB', 'GDP_perCapita_UN', 'Urea_Price', 'P_rock_Price', 'K2O_Price', 'Crop_Global_Price', 'Country_total_N', 'Country_total_P2O5', 'Country_total_K2O', 'Area_Irrig', 'Edu_Exp', 'pop_pressure', 'Avg_Size_Hold', 'Avg_Size_Hold_stand', 'MAP', 'N_removal', 'P_removal', 'K_removal', 'N_removal_ha', 'P_removal_ha', 'K_removal_ha', 'Avg_sqKm', 'K2O_perc_area', 'P_cost', 'N_cost', 'K_cost', 'soil_clay', 'soil_ph', 'soil_sand', 'soil_silt', 'soil_cec', 'soil_nitrogen', 'soil_ocs', 'AI', 'K2O_ha_cropland', 'P2O5_ha_cropland', 'N_ha_cropland', 'PET', 'TMN', 'Crop_Price_Real', 'Crop_Price_Nominal', 'n_mac_ag', 'Crop_Code_1_1', 'Crop_Code_1_2', 'Crop_Code_1_3', 'Crop_Code_1_4', 'Crop_Code_2_1', 'Crop_Code_2_2', 'Crop_Code_2_3', 'Crop_Code_3_1', 'Crop_Code_3_2', 'Crop_Code_4', 'Crop_Code_5', 'Crop_Code_6', 'Crop_Code_7', 'Region_Name_Australia and New Zealand', 'R

In [6]:
full_dataset = data_preprocessing.load_all_data(config=config)

for estimator_name, estimator_cls in estimators.items():
    if 'Hist' in estimator_name:
        print(estimator_name)
        estimator = estimator_cls(outpath=resultspath)
        estimator = estimator.load(path=estimator.output_path, it=0)  
        y_pred = make_prediction(estimator, full_dataset, config)

HistGradientBoostRegressor


In [7]:
y_pred

0         15.526538
1          5.672649
2         36.002052
3         22.684568
4          3.375614
            ...    
150018    76.304565
150019    76.973701
150020    73.475768
150021    73.595969
150022    62.826732
Name: predicted_K2O_avg_app, Length: 150023, dtype: float64

In [8]:
full_dataset.shape

(150023, 51)

In [11]:
# add the predictions to the full dataset
# full_dataset = full_dataset.join(y_pred)
it = 0
full_dataset.to_csv(resultspath / f'{target}_{it}_predictions.csv')

In [ ]:
# TODO: for the predictions, we need to combine them from the two folds somehow
# we could do this by taking the prediction from folds where samples were test samples
# for those not in the train/test set, we take the average of the two predictions
# or weigh it using the r2/rmse/... of the two predictions